 <div style="display:inline-block; position:absolute; top:0; right:0; color: #230e3e; font:italic 1em serif;">$\LaTeX$<span style="display:inline-block; vertical-align:bottom">
    <a href="https://www.mathjax.org">
    <img style="height:1.5em" title="Powered by MathJax"
    src="https://www.mathjax.org/badge/mj_logo.png"
    border="0" alt="Powered by MathJax" >
</a></span></div>
<div style="display:inline-block; position:absolute; top:0; left:0; color: #230e3e; font:italic 1em serif;"><a href="https://www.julialang.org">
    <img style="height:2.25em; vertical-align:top" title="Powered by Julia"
    src="./figures/julia-logo-mask.png"
    border="0" alt="Powered by Julia" >

</a></div>$\require{cancel} \require{mhchem} \require{bbox} \require{color} \require{enclose} \require{AMSmath} \require{AMSsymbols}$ 



In [12]:
## Jupyter notebook housekeeping
HTML("<link rel='stylesheet' href='./css/youcan-how-your-grade-is-calculated.css' type='text/css' charset='utf-8' />")

HTML{String}("<link rel='stylesheet' href='./css/youcan-how-your-grade-is-calculated.css' type='text/css' charset='utf-8' />")

In [11]:
using Plots
plotly();

# How your grade is calculated.

Your grade is a weighted average of your Tests, Homework, and Final Exam scores, as follows:  


\begin{gather}  \notag
 \textbf{GRADE} = 50 \times \textbf{TEST} + 25 \times \textbf{HOMEWORKS} + 25 \times \textbf{FINAL}
\end{gather}

Here is how it works, with a demo example. Farther down the page are live cells that you can use with your numbers.  
 
**HOMEWORKS** comes straight from WileyPLUS. `homeworks = 0.98`
```julia
homeworks = 0.98
```
**FINAL** is self-explanatory.
```julia
final = 0.95
```
**TESTS** requires a few intermediate calculations:
```julia
T = [83; 71; 89; 97] / 100
```
1. This drops the lowest test score, then calcuates the average.  
$ \qquad \small
\begin{gather} \notag \\ \notag
\textbf{DROP} = \frac{\text{sum} \left\{T_1, T_2, T_3, T_4 \right\} - \text{minimum} \left\{T_1, T_2, T_3, T_4 \right\}}{3}
\end{gather}
$  
```julia
drop = (sum(T) - minimum(T)) / 3
```
1. This replaces the lowest test score with FINAL if FINAL > lowest score, then calcuates the average. 
$ \qquad \small
\begin{gather} \notag \\ \notag
\textbf{REPLACE} = \frac{\text{sum} \left\{T_1, T_2, T_3, T_4, \textbf{FINAL} \right\} - \text{minimum} \left\{T_1, T_2, T_3, T_4,\textbf{FINAL} \right\}}{4}
\end{gather} 
$  
```julia
replace = (sum([T; final]) - minimum([T; final]))/4  
```
1. Wrapping up,  
$ \qquad \small
\begin{gather} \notag \\ \notag
\textbf{TESTS} \text{ = maximum\{} \textbf{DROP, REPLACE} \}
\end{gather}
$  
```julia
tests = max(drop, replace)
```

**GRADE** incorporates the weighted average.  
$ \qquad \small
\begin{gather} \notag \\ \notag
 \textbf{GRADE} = 50 \times \textbf{TEST} + 25 \times \textbf{HOMEWORKS} + 25 \times \textbf{FINAL}
\end{gather}
$ 
```julia
grade = 25*homeworks + 50*tests + 25*final
```

**LETTER GRADE** comes from this lookup table.  

score|97|92|90|87|82|80|77|72|70|67|62|60|0
-----|--|--|--|--|--|--|--|--|--|--|--|--|-
grade|A+|A |A-|B+|B |B-|C+|C |C-|D+|D |D-|F




<hr class="little" />

In [10]:
# MUST execute this cell before looking up a letter grade!

"""
the good
"""
function letter_grade(score)
    lg_map = [97 "A+"; 92 "A"; 90 "A-"; 87 "B+"; 82 "B"; 80 "B-"; 77 "C+"; 
        72 "C"; 70 "C-"; 67 "D+"; 62 "D"; 60 "D-"; 0 "F"]
    (lg_map[findfirst(floor(score+0.5) .>= lg_map[:,1] ),2] , 
        numberline(score,lg_map) )
end


"""
the bad
"""
function composite(homeworks,final,T)
    length(T) > 1 || println(                       # if there's only one test, print message, exit
        "error! mininum 2 tests req'd. ",
        "no substituton performed")    
    drop = (sum(T) - minimum(T)) / (length(T)-1)                 # average ⋮ largest n-1 scores, where n = # of tests
    replace = (sum([T; final]) - minimum([T; final]))/length(T)  # average 
    tests = max(drop, replace)
    
    # calculate the answer, rounded to 3 decimals
    floor((25*homeworks + 50*tests + 25*final) * 1000 + 0.5) / 1000
end


"""
the ugly
"""
function numberline(score,lg_map)
    scatter([grade],[0.2], 
        yaxis = false,
        size=(600,150),
        ylim=(0,0.45),
        markershape=:circle,
        legend=:none,
        markeralpha = 0.15,
        markerstrokealpha = 1,
        markercolor = :red,
        markerstrokecolor = :red)

    scatter!(lg_map[1:end-1,1],ones(size(lg_map,1)-1)*0.4,
        xticks = lg_map[1:end-1,1], 
    #     annotations = lg_map[1:end-1,2]), 
        marker=0.01,
        grid=:off,
        tickfontsize=9,
        aspect_ratio=5)

    annotate!(lg_map[1:end-1,1], ones(size(lg_map,1)-1)*0.45, 
        lg_map[1:end-1,2],9)
end
;

<hr class="big" />

In [13]:
homeworks = 0.8
final = 0.88
T = [0.7, 0.7, 0.7, 0.7]

grade = composite(homeworks,final,T)
(letter , nmbline) = letter_grade(grade)

display(nmbline)
display(HTML("<div class='verdict'> Numeric Score: $grade => $letter </div>"))

HTML{String}("<div class='verdict'> Numeric Score: 79.25 => C+ </div>")

<footer style="background-color: #461d7c; color:rgba(253,208,35,0.35); line-height:2em; padding-left:1em; padding-right:0.5em; bottom:0; float:bottom; text-align:right;"><i>ψᵟ</i></footer>